In [1]:
#%load_ext nb_black

In [2]:
import sys

sys.path.append("../")
import yaml
import logging
import requests
from pprint import pprint
import json
import time
import os
from pymongo import MongoClient
import dateparser
import psutil
from nlp_utils import NLPUtils
from lda_module import LdaModule

In [3]:
mongourl = "mongodb://localhost:27017/"
MONGO_CLIENT = MongoClient(mongourl)

In [4]:
collection = MONGO_CLIENT["news"]["article_en"]
not_processed_docs = collection.find()
# for doc in not_processed_docs:
#    print(doc)
#    time.sleep(10)

In [5]:
training_set = []
i = 0
for document in not_processed_docs:
    if i % 10000 == 0:
        print(i)
        print(psutil.virtual_memory())
    training_set.append(document['text'])
    i += 1

0
svmem(total=8589934592, available=3467591680, percent=59.6, used=4421881856, free=753553408, active=1808277504, inactive=2334334976, wired=2613604352)
10000
svmem(total=8589934592, available=2960384000, percent=65.5, used=4627853312, free=239079424, active=2078597120, inactive=2340159488, wired=2549256192)
20000
svmem(total=8589934592, available=2569011200, percent=70.1, used=4773064704, free=29667328, active=2288259072, inactive=2361139200, wired=2484805632)
30000
svmem(total=8589934592, available=2233610240, percent=74.0, used=4494913536, free=195264512, active=2041176064, inactive=1966297088, wired=2453737472)
40000
svmem(total=8589934592, available=2159792128, percent=74.9, used=4657033216, free=28495872, active=2135535616, inactive=2049372160, wired=2521497600)
50000
svmem(total=8589934592, available=2078982144, percent=75.8, used=4548829184, free=86376448, active=1995329536, inactive=1936846848, wired=2553499648)
60000
svmem(total=8589934592, available=1931239424, percent=77.5,

In [ ]:
# Load custom stop words from file
with open('custom_stop_words.json', 'r') as sw:
    custom_stop_words = json.load(sw)
custom_stop_words = custom_stop_words['s_words']

In [6]:
nlp_utils = NLPUtils(lang='en', custom_stop_words = custom_stop_words)

In [7]:
# Some preparation before running LDA
doc_collection = []
print("Parsing articles...")
i = 0
for doc in training_set[:1000]:
    if i % 100 == 0: print(i)
    tokens = nlp_utils.parse_text(doc)
    doc_collection.append(tokens)
    i += 1

Parsing articles...
0
100
200
300
400
500
600
700
800
900


In [8]:
print("Completed parsing {} articles".format(len(doc_collection)))

num_docs = len(doc_collection)
num_topics = 20
lda = LdaModule(lang = 'en', num_docs = num_docs, doc_collection = doc_collection, num_topics = num_topics, trained = False)
lda.runLDA()

docs_topics_dict = lda.get_docs_topics_dict()

Completed parsing 1000 articles
... Building dictionary ...
... Finding collocations ...
... Building corpus ...
... Building LDA model ...
... Retrieving topics ...


In [9]:
# == Saving model checkpoint ==
lda.save_LDA_model()